# Exercise 3: Predicting Customer Spend - Part 1
* Cho dữ liệu **retail_transactions.csv** _(chưa dc tiền xử lí)_ bao gồm các thông tin như sau:
  ![](../images/19.jpg)
  Hãy chuẩn hóa dữ liệu này và lưu vào tập tin **wrangled_transaction.csv**, gồm các thông tin như sau:
  ![](../images/20.jpg)
* Thông tin cung cấp:
  * Dataset này chứa các lịch sử giao dịch từ năm 2010 đến 2011. Với mỗi giao dịch có chứa **customer identifier** _(CustomerID)_, **số lượng hàng đã mua** _(Quantity)_, **ngày mua** _(InvoiceDate)_, **đơn giá** _(Unitprice)_ và một số thông tin khác về mặt hàng đã mua.
  * Cần tiền xử lí dữ liệu này thành dữ liệu giao dịch của khách hàng từ năm 2010 sau đó so với chỉ tiêu năm 2011. Vậy nên cần tạo ra các feature từ dữ liệu cho năm 2010 và tính toán số tiền đã chi cho năm 2011.
  * Khi xây dựng mô hình này, nó sẽ khái quất cho những năm tới. Nhờ đó doanh nghiệp có thể sử dụng dữ liệu năm 2020 để dự đoán trc hành vi chi tiêu vào năm 2021 _(trừ khi thị trường hoặc doanh nghiệp đã thay đổi đáng kể kể từ khoảng thời gian dữ liệu dc sử dụng để build model)_.

**Đọc dữ liệu, tìm hiểu thông tin từ dữ liệu**

In [1]:
import pandas as pd
import pandas_profiling as pp

In [2]:
df = pd.read_csv(r'./data/retail_transactions.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,546729,22775,PURPLE DRAWERKNOB ACRYLIC EDWARDIAN,12,2011-03-16 11:36:00,1.25,18231.0,United Kingdom
1,559898,21868,POTTING SHED TEA MUG,6,2011-07-13 12:18:00,1.25,16225.0,United Kingdom
2,548648,71459,HANGING JAM JAR T-LIGHT HOLDER,24,2011-04-01 13:20:00,0.85,12949.0,United Kingdom
3,540543,22173,METAL 4 HOOK HANGER FRENCH CHATEAU,4,2011-01-09 15:23:00,2.95,14395.0,United Kingdom
4,561390,20726,LUNCH BAG WOODLAND,10,2011-07-27 09:52:00,1.65,17068.0,United Kingdom


In [3]:
df.shape

(397884, 8)

In [4]:
# profile = pp.ProfileReport(df)
# profile

**Chuyển đổi dữ liệu cột `InvoiceDate` thành định dạng `datetime`**

In [5]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,546729,22775,PURPLE DRAWERKNOB ACRYLIC EDWARDIAN,12,2011-03-16 11:36:00,1.25,18231.0,United Kingdom
1,559898,21868,POTTING SHED TEA MUG,6,2011-07-13 12:18:00,1.25,16225.0,United Kingdom
2,548648,71459,HANGING JAM JAR T-LIGHT HOLDER,24,2011-04-01 13:20:00,0.85,12949.0,United Kingdom
3,540543,22173,METAL 4 HOOK HANGER FRENCH CHATEAU,4,2011-01-09 15:23:00,2.95,14395.0,United Kingdom
4,561390,20726,LUNCH BAG WOODLAND,10,2011-07-27 09:52:00,1.65,17068.0,United Kingdom


**Tạo cột `revenue` = `UnitPrice` * `Quantity`**

In [6]:
df['revenue'] = df['UnitPrice']*df['Quantity']
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
0,546729,22775,PURPLE DRAWERKNOB ACRYLIC EDWARDIAN,12,2011-03-16 11:36:00,1.25,18231.0,United Kingdom,15.0
1,559898,21868,POTTING SHED TEA MUG,6,2011-07-13 12:18:00,1.25,16225.0,United Kingdom,7.5
2,548648,71459,HANGING JAM JAR T-LIGHT HOLDER,24,2011-04-01 13:20:00,0.85,12949.0,United Kingdom,20.4
3,540543,22173,METAL 4 HOOK HANGER FRENCH CHATEAU,4,2011-01-09 15:23:00,2.95,14395.0,United Kingdom,11.8
4,561390,20726,LUNCH BAG WOODLAND,10,2011-07-27 09:52:00,1.65,17068.0,United Kingdom,16.5


**Quan sát thấy rằng mỗi hóa đơn dc trải đều trên nhiều hàng, mỗi hàng là một loại sản phẩm dc mua $\Rightarrow$ cần nhóm dữ liệu lại sao cho mỗi `InvoiceNo` chỉ nằm trên duy nhất một hàng với các feature đi theo là `revenue`, `InvoiceDate` và `CustomerID`.**

In [7]:
operations = {
    'revenue': 'sum',
    'InvoiceDate': 'first',
    'CustomerID': 'first'
}

df = df.groupby('InvoiceNo').agg(operations)
df.head()

,revenue,InvoiceDate,CustomerID
InvoiceNo,,,
536365,139.12,2010-12-01 08:26:00,17850.0
536366,22.20,2010-12-01 08:28:00,17850.0
536367,278.73,2010-12-01 08:34:00,13047.0
536368,70.05,2010-12-01 08:34:00,13047.0
536369,17.85,2010-12-01 08:35:00,13047.0


In [8]:
df.shape

(18532, 3)

> **Nhận xét**
> * Từ dữ liệu ban đầu bao gồm 397884 quan sát, bây giờ chỉ còn 18532 quan sát.

**Vì ta cần sử dụng _năm_ để quyết định để quyết định quan sát nào sẽ dc đưa vào tập input $\Rightarrow$ tạo ra thêm một thuộc tính là `year`**

In [9]:
df['year'] = df['InvoiceDate'].apply(lambda x: x.year)

df.head()

,revenue,InvoiceDate,CustomerID,year
InvoiceNo,,,,
536365,139.12,2010-12-01 08:26:00,17850.0,2010
536366,22.20,2010-12-01 08:28:00,17850.0,2010
536367,278.73,2010-12-01 08:34:00,13047.0,2010
536368,70.05,2010-12-01 08:34:00,13047.0,2010
536369,17.85,2010-12-01 08:35:00,13047.0,2010


**Ngày thực hiện giao dịch cũng có thể là một feature quan trọng, thông tin về khoảng thời gian từ lần cuối khách hàng mua hàng so với cuối năm và lần đầu khách hàng mua hàng vào năm sau so với cuối năm trước cũng có thể là những thông tin cực kì hữu ích, nên cần tính các khoảng cách thời gian này trên mỗi hóa đơn.**

In [10]:
df['days_since'] = (pd.datetime(year=2010, month=12, day=31) - df['InvoiceDate']).apply(lambda x: x.days)

df.head()

<ipython-input-10-5d9ec0479043>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df['days_since'] = (pd.datetime(year=2010, month=12, day=31) - df['InvoiceDate']).apply(lambda x: x.days)


,revenue,InvoiceDate,CustomerID,year,days_since
InvoiceNo,,,,,
536365,139.12,2010-12-01 08:26:00,17850.0,2010,29
536366,22.20,2010-12-01 08:28:00,17850.0,2010,29
536367,278.73,2010-12-01 08:34:00,13047.0,2010,29
536368,70.05,2010-12-01 08:34:00,13047.0,2010,29
536369,17.85,2010-12-01 08:35:00,13047.0,2010,29


In [11]:
df.tail()

,revenue,InvoiceDate,CustomerID,year,days_since
InvoiceNo,,,,,
581583,124.60,2011-12-09 12:23:00,13777.0,2011,-344
581584,140.64,2011-12-09 12:25:00,13777.0,2011,-344
581585,329.05,2011-12-09 12:31:00,15804.0,2011,-344
581586,339.20,2011-12-09 12:49:00,13113.0,2011,-344
581587,249.45,2011-12-09 12:50:00,12680.0,2011,-344


**Hiện tại dữ liệu đang dc nhóm theo `InvoiceNo`, nhưng chúng ta cần một dataframe khác _(tạm gọi là X)_ dc nhóm theo `CustomerID`, tại đây ta cần thống kê tổng số tiền mà khách hàng này đã chi ra, khoảng thời gian lâu nhất mà khách hàng này ko mua hàng, và khoảng thời gian ngắn nhất mà khách hàng quay lại mua hàng lần nữa so với lần trc đó, đồng thời cần đếm ra có bao nhiêu lần khách hàng quay lại mua hàng.**

In [12]:
operations = {
    'revenue': 'sum',
    'days_since': ['max', 'min', 'nunique']
}

X = df[df['year'] == 2010].groupby('CustomerID').agg(operations)

X.head()

revenue days_since            
                sum        max min nunique
CustomerID                                
12347.0      711.79         23  23       1
12348.0      892.80         14  14       1
12370.0     1868.02         16  13       2
12377.0     1001.52         10  10       1
12383.0      600.72          8   8       1

In [13]:
X.columns = [' '.join(col).strip() for col in X.columns.values]
X.head()

,revenue sum,days_since max,days_since min,days_since nunique
CustomerID,,,,
12347.0,711.79,23,23,1
12348.0,892.80,14,14,1
12370.0,1868.02,16,13,2
12377.0,1001.52,10,10,1
12383.0,600.72,8,8,1


**Tạo thêm một feature mới là `avg_order_cost` _(chi tiêu trung bình cho mỗi đơn hàng)_ bằng cách chia `revenue sum` _(tổng số tiền mà một khách hàng chi ra để mua hàng)_ cho `days_since nunique`.**

In [14]:
X['avg_order_cost'] = X['revenue sum'] / X['days_since nunique']

X.head()

,revenue sum,days_since max,days_since min,days_since nunique,avg_order_cost
CustomerID,,,,,
12347.0,711.79,23,23,1,711.79
12348.0,892.80,14,14,1,892.80
12370.0,1868.02,16,13,2,934.01
12377.0,1001.52,10,10,1,1001.52
12383.0,600.72,8,8,1,600.72


**Bây giờ, ta đã có các feature input, chúng ta cần kết quả output, đó là tổng doanh thu cho năm 2011 _(ta sẽ tạo một thuộc tính là `revenue_2011`)_ bằng cách nhóm theo `CustomerID` và cộng các giá trị `revenue` lại, cuối cùng ta lưu nó vào biến `y`.**

In [15]:
y = df[df['year'] == 2011].groupby('CustomerID')['revenue'].sum()

y

CustomerID
12346.0    77183.60
12347.0     3598.21
12348.0      904.44
12349.0     1757.55
12350.0      334.40
             ...   
18280.0      180.60
18281.0       80.82
18282.0      178.05
18283.0     2094.88
18287.0     1837.28
Name: revenue, Length: 4219, dtype: float64